# Ricostruire la divina commedia con le catene di Markov

Il nostro scopo è ricostruire la struttura del DNA.

Il problema è che non sappiamo cosa stiamo cercando!

Come possiamo capire se e quanto i nostri metodi funzionano in modo appropriato, e quali features catturano?

Per fare questo, possiamo usare un dataset di prova, che conosciamo bene, per testare se e quanto il nostro metodo sia affidabile e preciso.

Nel nostro caso, useremo la divina commedia come sostituto del DNA.

* sappiamo riconoscere al volo suoni simili all'italiano
* sappiamo distinguere italiano moderno da quello antico
* sappiamo che ha strutture quasi locali (endecasillabo, rima alternata) e così via.

Possiamo testare uno o più modelli per capire come e quanto riproduce queste caratteristiche.

Useremo una versione della divina commedia fornita gratuitamente dal [progetto Gutenberg](https://www.gutenberg.org/).

Ho rimosso tutto il testo che non è la divina commedia, inclusi i titoli dei vari canti.

Questo sarà il nostro testo di riferimento

Vogliamo unire tutto il testo in un unico flusso di caratteri stampabili.

Per far questo ci dobbiamo appoggiare alla libreria di python chiamata *itertools* che ci fornisce gli strumenti per manipolare gli iteratori

In [1]:
import itertools as it

useremo due funzioni principali:
    
* **itertools.chain.from_iterable** per combinare le linee in un flusso unico
* **itertools.islice** per selezionare soltanto una parte del nostro testo invece che tutto

## itertools.chain.from_iterable

In [2]:
lista_di_liste = [[1, 2], [3, 4]]
for elemento in lista_di_liste:
    print(elemento)

[1, 2]
[3, 4]


In [3]:
for elemento in it.chain.from_iterable(lista_di_liste):
    print(elemento)

1
2
3
4


## itertools.islice

In [4]:
lista_lunga = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

for elemento in it.islice(lista_lunga, 5):
    print(elemento)

1
2
3
4
5


Per le liste potrei farlo anche in modo più semplice, ma in generale gli iteratore non mi supportano la sottoselezione in modo semplice.

In [5]:
file = './divinacommedia_cleaned.txt'
# devo leggere il file nel suo encoding, in questo caso non standard
with open(file, 'r', encoding='utf-8-sig') as testo:
    testo = it.chain.from_iterable(testo)
    head = it.islice(testo, 50)
    result = list(head)
    print(result)

['N', 'e', 'l', ' ', 'm', 'e', 'z', 'z', 'o', ' ', 'd', 'e', 'l', ' ', 'c', 'a', 'm', 'm', 'i', 'n', ' ', 'd', 'i', ' ', 'n', 'o', 's', 't', 'r', 'a', ' ', 'v', 'i', 't', 'a', '\n', 'm', 'i', ' ', 'r', 'i', 't', 'r', 'o', 'v', 'a', 'i', ' ', 'p', 'e']


Per rendere più semplice la nostra analisi, convertiamo tutto in minuscolo, in modo che le maiuscole non vengano viste come lettere differenti.

In [6]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    head = it.islice(testo, 50)
    result = list(head)
    print(result)

['n', 'e', 'l', ' ', 'm', 'e', 'z', 'z', 'o', ' ', 'd', 'e', 'l', ' ', 'c', 'a', 'm', 'm', 'i', 'n', ' ', 'd', 'i', ' ', 'n', 'o', 's', 't', 'r', 'a', ' ', 'v', 'i', 't', 'a', '\n', 'm', 'i', ' ', 'r', 'i', 't', 'r', 'o', 'v', 'a', 'i', ' ', 'p', 'e']


Ora iniziamo con il nostro modello più semplice: generiamo il testo semplicemente ripetendo le lettere in base a quanto sono frequenti.

Questo ci richiede per prima cosa di valutare la frequenza di queste lettere!

Per far questo, abbiamo uno strumento giù predisposo, la classe **Counter**.

In [7]:
from collections import Counter

lettere = "aaaabb"
conteggi = Counter(lettere)
conteggi.most_common()

[('a', 4), ('b', 2)]

In [8]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    head = it.islice(testo, 50)
    conteggi = Counter(head)
    print(conteggi)

Counter({' ': 8, 'i': 6, 'e': 4, 'm': 4, 'a': 4, 'n': 3, 'o': 3, 't': 3, 'r': 3, 'l': 2, 'z': 2, 'd': 2, 'v': 2, 'c': 1, 's': 1, '\n': 1, 'p': 1})


ora vogliamo estrarre le lettere a caso in modo proporzionale a quanto le abbiamo visto di frequente.

Per far questo, possiamo usare la funzione **choices** della libreria **random**.

In [9]:
from random import choices

lettere = ['a', 'b']
frequenze = [9, 1]

choices(lettere, weights=frequenze, k=10)

['a', 'a', 'a', 'a', 'a', 'a', 'b', 'a', 'a', 'a']

se volessimo usarlo per generare il testo, possiamo unire tutte queste lettere con un join

In [10]:
random = choices(lettere, weights=frequenze, k=10)
testo = str.join('', random)
print(testo)

aaaaaaaaaa


Possiamo usare **choices** in combinazione con **Counter** in modo semplice

In [11]:
lettere = "aaaabb"
conteggi = Counter(lettere)
print(conteggi.keys(), conteggi.values())

dict_keys(['a', 'b']) dict_values([4, 2])


In [12]:
lettere = "aaaabb"
conteggi = Counter(lettere)
lettere = list(conteggi.keys())
frequenze = list(conteggi.values())
random = choices(lettere, weights=frequenze, k=20)
testo = str.join('', random)
print(testo)

bbaaabaaaabaaabbaaba


In [13]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    head = it.islice(testo, 100)
    conteggi = Counter(head)
    
lettere = list(conteggi.keys())
frequenze = list(conteggi.values())
random = choices(lettere, weights=frequenze, k=100)
testo = str.join('', random)
print(testo)

ioélvtsal rlai cia l m  siszzlee riizesmuri   iartaled omaaiméazarneavru avrrsamr mvrt, usm  ar aaaa


Il risultato è chiaramente deludente.

Non solo non assomiglia ad una frase italiana...non sembrano neppure parole!

Cosa sta succedendo?


Sappiamo che nell'italiano le parole non sono composte semplicemente dai suoni, ma da come i suoni i susseguono l'un l'altro.

Per rappresentare questa conoscenza, possiamo utilizzare un metodo della fisica matematica chiamato Catene di Markov.

## Le catene di Markov

L'idea è semplice: invece di generare ogni lettere sulla base della sua frequenza nel testo, facciamo un passaggio leggermente più intelligente.

* Guardo l'ultima lettera del testo che ho generato finora.
* Considero nel testo originale quanto spesso ciascuna lettera segue la lettera che ho scelto.
* Genero la mia nuova lettera sulla base di questa **probabilità condizionata**.
* Ricomincio usando l'ultima lettera generata come punto di partenza.

Questo metodo è lo stesso (di base) usato dai cellulari per il suggerimento delle parole.

Se scrivete spesso "buona notte", "buona mattina" e "buona cena", una volta che inserite la parola "buona" da tastiera, il software sa che probabilmente dopo andrete ad inserire "notte", "mattina" o "cena".

Se non ha a disposizione un buon campione personalizzato, utilizza quelli delle persone della stessa lingua.

Questo ha creato non poco imbarazzo nel caso di frasi volgari o razziste, inserite dall'algoritmo come proposte per via del loro uso frequente (vedi il caso recente dell'IPhone che suggeriva la frase "Vesuvio lavali col fuoco")

Per ciascuna lettera dell'alfabeto dovremo quindi avere un **Counter** che mi tenga conto di quali lettere potranno seguire.

Per poter fare questo devo unire le lettere del mio testo in coppie consecutive di lettere!

Per fare questo uso una libreria esterna, **toolz**, che mi fornisce la funzione **sliding_window** che fa proprio questo

In [14]:
from toolz import sliding_window

lettere = "abcdefg"

coppie = sliding_window(2, lettere)

print(list(coppie))

[('a', 'b'), ('b', 'c'), ('c', 'd'), ('d', 'e'), ('e', 'f'), ('f', 'g')]


## WARNING

Useremo degli algoritmi assolutamente non ottimali!

In questo caso ci servono solo per capire il procedimento, nella vita reale sono troppo lenti per funzionare davvero!

In [15]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    head = it.islice(testo, 30)
    coppie = sliding_window(2, head)
    print(list(coppie))

[('n', 'e'), ('e', 'l'), ('l', ' '), (' ', 'm'), ('m', 'e'), ('e', 'z'), ('z', 'z'), ('z', 'o'), ('o', ' '), (' ', 'd'), ('d', 'e'), ('e', 'l'), ('l', ' '), (' ', 'c'), ('c', 'a'), ('a', 'm'), ('m', 'm'), ('m', 'i'), ('i', 'n'), ('n', ' '), (' ', 'd'), ('d', 'i'), ('i', ' '), (' ', 'n'), ('n', 'o'), ('o', 's'), ('s', 't'), ('t', 'r'), ('r', 'a')]


In [16]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    head = it.islice(testo, 300)
    coppie = sliding_window(2, head)
    conteggi = Counter(coppie)
    print(conteggi.most_common(5))

[(('a', ' '), 18), (('r', 'a'), 9), (('e', 'l'), 7), ((' ', 'd'), 7), (('t', 'a'), 6)]


quante combinazioni posso avere?

per semplificare, consideriamo solo le lettere, gli spazi e le lettere accentate. 

Otteniamo circa 30 caratteri.

tutte le coppie sono date da $30^2 = 900$

Se tutte queste coppie fossero equiprobabili e ne volessimo avere almeno 10 campionamenti per coppia, dovremmo avere circa 10'000 lettere.

Quando useremo le triplette, avremo $30^3 = 27'000$ combinazioni, quindi avremo bisogno di circa 300'000 lettere per avere una buona copertura.

quante lettere abbiamo in totale nella nostra divina commedia?

In [17]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    conteggio = sum(1 for c in testo)
print(conteggio)

529770


Questo ci dice che non potremo davvero andare oltre le triplette, almeno senza renderci le cose difficili.

In [18]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    coppie = sliding_window(2, testo)
    conteggi = Counter(coppie)
print(len(conteggi))
print(conteggi.most_common(5))

855
[(('e', ' '), 17397), (('a', ' '), 13949), (('i', ' '), 11636), (('o', ' '), 10906), ((' ', 'c'), 9579)]


possiamo vedere come in realtà, anche con le sole coppie, non riusciamo a coprire tutti i casi possibili!

Però iniziamo già a vedere una prima feature della lingua italiana: le parole finiscono spesso con delle vocali.

Vediamo ora come possiamo sfruttare questo dizionario di conteggi per generare il nostro testo.

Dobbiamo partire da un seme, un elemento di testo iniziale.
Nel nostro caso, per cercare di rimanere il più generici possibile, useremo uno spazio vuoto.

per poter generare la nuova lettera a partire dalla vecchia, dovremo selezionare soltanto quella parte del nostro dizionario che contiene la nostra lettera come parte iniziale.

Possiamo usare la funzione **item** dei dizionari per avere la sequenza delle coppie chiave valore, e filtrarle in base al loro punto di inizio

In [19]:
dizionario = {'a':1, 'b':2, 'c':3}
print(dizionario.items())

dict_items([('a', 1), ('b', 2), ('c', 3)])


In [20]:
chiavi = [c for c, v in dizionario.items() if c in ['b', 'c']]
valori = [v for c, v in dizionario.items() if c in ['b', 'c']]
print(chiavi, valori)

['b', 'c'] [2, 3]


ora possiamo fare lo stesso con il nostro vettore dei conteggi

In [23]:
nuovo_testo = [' ']
for i in range(10):
    lettera = nuovo_testo[-1]
    lettere = [c[1] for c, v in conteggi.items() if c[0]==lettera]
    frequenze = [v for c, v in conteggi.items() if c[0]==lettera]
    prossima_lettera = choices(lettere, frequenze)[0]
    print(lettera, prossima_lettera)

  r
  e
  e
  n
  p
  s
  l
  n
  a
  c


ci siamo dimenticati di attaccare l'ultima lettera generata al nostro testo!

In [24]:
nuovo_testo = [' ']
for i in range(10):
    lettera = nuovo_testo[-1]
    lettere = [c[1] for c, v in conteggi.items() if c[0]==lettera]
    frequenze = [v for c, v in conteggi.items() if c[0]==lettera]
    prossima_lettera = choices(lettere, frequenze)[0]
    print(lettera, prossima_lettera)
    nuovo_testo.append(prossima_lettera)

  s
s t
t e
e f
f r
r i
i s
s a
a i
i  


In [25]:
print(str.join('', nuovo_testo))

 stefrisai 


proviamo ora un testo più lungo!

In [26]:
nuovo_testo = [' ']
for i in range(200):
    lettera = nuovo_testo[-1]
    lettere = [c[-1] for c, v in conteggi.items() if c[0]==lettera]
    frequenze = [v for c, v in conteggi.items() if c[0]==lettera]
    prossima_lettera = choices(lettere, frequenze)[0]
    nuovo_testo.append(prossima_lettera)
print(str.join('', nuovo_testo))

 e gh’a vesta
ocheïom’ sercosone mirenadietre coma cche ce iove co,
ino,
«l sio vogue cenel’i no, co.

ndido, me geioniar ’e e,
sónoranve: rotosi da ttr,
ere si ’ de, quielò de ato ce l ’a a chi sediru


il risultato lascia ancora a desiderare...

proviamo ad estendere alle triplette: in questo caso consideriamo le coppie di lettere come punto di partenza e la nuova lettera sulla base di queste.

Dobbiamo però ricostruire il nostro intero dataset dei conteggi.

In [27]:
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    coppie = sliding_window(3, testo)
    conteggi = Counter(coppie)
print(len(conteggi))
print(conteggi.most_common(5))

6059
[((' ', 'c', 'h'), 4036), (('c', 'h', 'e'), 3871), (('h', 'e', ' '), 3630), ((' ', 'd', 'i'), 3338), (('.', '\n', '\n'), 3171)]


il procedimento per generare il testo è assolutamente identico, ma il controllo sarà sulle ultime due lettere e non soltanto sull'ultima.

In [28]:
nuovo_testo = ['c', 'h']
for i in range(200):
    lettera = tuple(nuovo_testo[-2:])
    lettere = [c[-1] for c, v in conteggi.items() if c[:2]==lettera]
    frequenze = [v for c, v in conteggi.items() if c[:2]==lettera]
    prossima_lettera = choices(lettere, frequenze)[0]
    nuovo_testo.append(prossima_lettera)
print(str.join('', nuovo_testo))

ch’enducia».

el prisolo bia
penti a gel ché moda ti ger riposì ché ma va,
vernima.

d’e’ affine danime lormio
e una bullibisonda qualto io su posti;

mecomer cortintro va o el mo in don fro,
e lavellev


Possiamo facilmente estendere questo procedimento ad ancora più lettere.

Vedremo che con l'aumentare della lunghezza della sequenza avrò parole sempre più simili all'italiano, ma il numero di casi disponibili calerà sensibilmente!

In [29]:
L = 3
file = './divinacommedia_cleaned.txt'
with open(file, 'r', encoding='utf-8-sig') as testo:
    linee = (linea.lower() for linea in testo)
    testo = it.chain.from_iterable(linee)
    coppie = sliding_window(L+1, testo)
    conteggi = Counter(coppie)
    
nuovo_testo = ['n', 'e', 'l']
for i in range(200):
    lettera = tuple(nuovo_testo[-L:])
    lettere = [c[-1] for c, v in conteggi.items() if c[:L]==lettera]
    frequenze = [v for c, v in conteggi.items() if c[:L]==lettera]
    prossima_lettera = choices(lettere, frequenze)[0]
    nuovo_testo.append(prossima_lettera)
print(str.join('', nuovo_testo))

nel sape
che ne proci coi tempresse noia è ’l sa il capo giava.

e ti biavorriga.

per che la donoso assame
esti,

e sona.

sol del ben sua già nullo la vista in piegatente dentì lonte ha mestorna,
e si 


posso generalizzare questo procedimento con una funzione per generare dei testi lunghi a piacere, usando una funzione.

In [30]:
def genera_testo(nuovo_testo='nel', lunghezza=2):
    L = lunghezza
    nuovo_testo = list(nuovo_testo)
    file = './divinacommedia_cleaned.txt'
    with open(file, 'r', encoding='utf-8-sig') as testo:
        linee = (linea.lower() for linea in testo)
        testo = it.chain.from_iterable(linee)
        coppie = sliding_window(L+1, testo)
        conteggi = Counter(coppie)

    while True:
        lettera = tuple(nuovo_testo[-L:])
        lettere = [c[-1] for c, v in conteggi.items() if c[:L]==lettera]
        frequenze = [v for c, v in conteggi.items() if c[:L]==lettera]
        prossima_lettera = choices(lettere, frequenze)[0]
        nuovo_testo.append(prossima_lettera)
        nuovo_testo = nuovo_testo[1:]
        yield prossima_lettera


In [31]:
testo = str.join("", it.islice(genera_testo('nel', 2), 200))
print(testo)

la on pubito a brente,
si quel bole ’ iordison dov’ebbelo ’l che,
cortarol sporre o pocci
antroscerte note chi fia bento né noncier che lassolà dun te ’l vida grista ocche quan pin sé pa,
e ’l fola là


In [32]:
testo = str.join("", it.islice(genera_testo('che', 2), 200))
print(testo)

rvi».

che poscuombe vutte tor vol, quante cembra si, almi combil conde ’l sarta
par disper ocoi monte mal crubilla sé purba
che ummi cor li a de haniglingel che perbia
e poia for né pere
che questrai


In [33]:
testo = str.join("", it.islice(genera_testo('nel', 3), 200))
print(testo)

 cinquesti là dolo una buonacquei divinanza, che ’l mi te era mi conti lena il vien: «più si dice acun: ‘i’ di mi dissai,
che noi lumeria, viso.

«quandome splendo,
sommi cosa compiogo.

poi cominci s


In [34]:
testo = str.join("", it.islice(genera_testo('nel ', 4), 200))
print(testo)

terra questo
tacito maria con quella a quindi povera, e te ne l’ha meritón creatura,
udirai, ma s’ebbe “i’ mi mosso, che la ripa usciole se in su le fischi crissimo fossi i più carno, per acqua, e fa 


In [120]:
testo = str.join("", it.islice(genera_testo('onore', 5), 200))
print(testo)


a la bifolco mi cacciati sù, mo nel ciel voto a dir di lei di parnaso,

e torre mostra maggio
de l’occhi se’ giocondo
s’adempia.

qui vedrai
cosa di comincia’ io di colui che vinse;

ma perché ’l buo


Possiamo vedere come il testo generato assomigli all'italiano in modo superficiale, ma le frasi non hanno senso.

Potremmo usare la nostra conoscenza dell'esistenza delle parole per generare delle frasi più sensate, ma saremmo sempre ciechi alle strutture di ordine superiore.